<h1 align=center><font size = 5>Comparing Neighborhoods of Newton, MA</font></h1>

# 1. Purpose & Introduction

This notebook provides a comparison between the different neighborhoods (also known as villages) in the city of Newton, MA. The comparison is used to help  families who want to move to Newton to choose a neighborhood that is best suited for their needs. Generally, a family will want to look at what different neighborhoods have to offers. For example, they may want to be close to public transit, they may want to have play grounds nearby if they have little kids. A couple without kids may want to have access to a lot of restaurants. A family also will consider housing price if they want to buy a house. 

It should be noted that for this project, the main interest is to focus on Newton, MA. This can be easily expanded to any other city as long as the data for such city is available. 

# 2. Data Source and Description

The data of interest for a family include nearby amenities/venues (data can be obtained from Foursquare) and housing prices, which will be obtained from the website Redfin, which has a lot of data on sold house price. The crime data of of the city will be obtained as well. 

The venues data from four square will be obtained to determine the top venues in a neighborhood. This data will help a family to see if the amenities in that neighborhood meet their needs. The house price data will help a family to determine if they can afford a house of interest in such neighborhood. The crime data help a family to determine how safe such a neighborhood is. 